In [1]:
# ---------- Importing Libraries ----------
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

# Display options
pd.set_option("display.max_columns", 50)



## Loading & Merging Datasets

In [2]:

# ---------- Paths ----------
data = Path(r"C:/Users/Aisha Kanyiti/Desktop/TMP 2nd Project")

# ---------- Helper: Check if file has header ----------
def file_has_header(path: Path, probe_prefix="timestamp"):
    with open(path, "r", encoding="utf-8") as f:
        first = f.readline().strip().strip("#").strip('"').lower()
    return first.startswith(probe_prefix)

# ---------- Load Category Tree (Raw) ----------
cat_tree_raw = pd.read_csv(data / "category_tree(in).csv", header=None, dtype=str, encoding="utf-8")


# ---------- Load Events (Raw) ----------
events_path = data / "events(in).csv"
events_has_header = file_has_header(events_path, probe_prefix="timestamp")
events_raw = pd.read_csv(
    events_path,
    header=0 if events_has_header else None,
    names=None if events_has_header else ["timestamp", "visitorid", "event", "itemid", "transactionid"],
    encoding="utf-8"
)

# ---------- Load Item Properties Parts (Raw) ----------
def load_item_props_raw(path: Path):
    has_header = file_has_header(path, probe_prefix="timestamp")
    return pl.read_csv(
        path,
        has_header=has_header,
        new_columns=None if has_header else ["timestamp", "itemid", "property", "value"]
    )

ip1_raw = load_item_props_raw(data / "item_properties_part1.1.csv")
ip2_raw = load_item_props_raw(data / "item_properties_part2.csv")

# Combine
item_properties_raw = pl.concat([ip1_raw, ip2_raw])

print("Item_properties_raw shape:", item_properties_raw.shape)


Item_properties_raw shape: (20275902, 4)


In [3]:
import os
print(os.listdir(data))


['.ipynb_checkpoints', '.venv', 'About Recommendation System Dataset.pdf', 'category_tree(in).csv', 'events(in).csv', 'item_properties_part1.1.csv', 'item_properties_part2.csv', 'project.py', 'Rec.ipynb', 'sys.ipynb']


Data Cleaning

In [4]:

# ---------- Clean Category Tree ----------
def clean_category_tree(df: pd.DataFrame) -> pd.DataFrame:
    def is_int_like(s):
        s = str(s).strip()
        return s.lstrip("-").isdigit()
    if df.shape[1] >= 2 and (not is_int_like(df.iloc[0, 0]) or not is_int_like(df.iloc[0, 1])):
        df = df.iloc[1:].copy()
    df = df.iloc[:, :2]
    df.columns = ["child", "parent"]
    df["child"] = pd.to_numeric(df["child"], errors="coerce").astype("Int64")
    df["parent"] = pd.to_numeric(df["parent"], errors="coerce").astype("Int64")
    return df.dropna(subset=["child", "parent"]).reset_index(drop=True)

cat_tree = clean_category_tree(cat_tree_raw)

# ---------- Clean Events ----------
def clean_events(df: pd.DataFrame) -> pd.DataFrame:
    for col in ["timestamp", "visitorid", "event", "itemid", "transactionid"]:
        if col in df.columns and df[col].dtype == "object":
            df[col] = df[col].astype(str).str.strip()
    df["timestamp"] = pd.to_numeric(df["timestamp"], errors="coerce")
    df["visitorid"] = pd.to_numeric(df["visitorid"], errors="coerce")
    df["itemid"] = pd.to_numeric(df["itemid"], errors="coerce")
    if "transactionid" in df.columns:
        df["transactionid"] = pd.to_numeric(df["transactionid"], errors="coerce")
    df["event"] = df["event"].astype("category")
    df = df.dropna(subset=["timestamp", "visitorid", "event", "itemid"]).reset_index(drop=True)
    df["timestamp"] = df["timestamp"].astype("int64")
    df["visitorid"] = df["visitorid"].astype("int64")
    df["itemid"] = df["itemid"].astype("int64")
    return df

events = clean_events(events_raw)

# ---------- Clean Item Properties ----------
# Remove 'available' and 'categoryid' rows (Polars syntax)
item_properties_filtered = item_properties_raw.filter(
    ~pl.col("property").is_in(["available", "categoryid"])
)

# Clean Item Properties
def clean_item_properties(df: pl.DataFrame) -> pl.DataFrame:
    if df["timestamp"].dtype != pl.Int64:
        df = df.with_columns(pl.col("timestamp").cast(pl.Int64, strict=False))
    if df["itemid"].dtype != pl.Int64:
        df = df.with_columns(pl.col("itemid").cast(pl.Int64, strict=False))
    df = df.with_columns([
        pl.col("property").cast(pl.Utf8),
        pl.col("value").cast(pl.Utf8)
    ])
    return df.sort(["itemid", "property", "timestamp"])

item_properties = clean_item_properties(item_properties_filtered)


# ---------- Sanity Checks ----------
print(f"Category tree: {cat_tree.shape}")
print(f"Events: {events.shape} | dtypes:\n{events.dtypes}")
print(f"Item properties (Polars): {item_properties.shape}")
print(item_properties.head(3))


Category tree: (1644, 2)
Events: (1048575, 5) | dtypes:
timestamp           int64
visitorid           int64
event            category
itemid              int64
transactionid     float64
dtype: object
Item properties (Polars): (17984049, 4)
shape: (3, 4)
┌───────────────┬────────┬──────────┬──────────────────────────────┐
│ timestamp     ┆ itemid ┆ property ┆ value                        │
│ ---           ┆ ---    ┆ ---      ┆ ---                          │
│ i64           ┆ i64    ┆ str      ┆ str                          │
╞═══════════════╪════════╪══════════╪══════════════════════════════╡
│ 1433646000000 ┆ 0      ┆ 1036     ┆ 1276750                      │
│ 1433041200000 ┆ 0      ┆ 1056     ┆ n3.168 1144008               │
│ 1433646000000 ┆ 0      ┆ 11       ┆ n15360.000 628176 n12288.000 │
└───────────────┴────────┴──────────┴──────────────────────────────┘


In [5]:
import polars as pl

# 1) Load events and category tree
events = pl.read_csv("events(in).csv")
cat_tree = pl.read_csv("category_tree(in).csv")

# 2) Load both item_properties files and combine
ip1 = pl.read_csv("item_properties_part1.1.csv")
ip2 = pl.read_csv("item_properties_part2.csv")

# Merge them vertically
item_properties_raw = pl.concat([ip1, ip2])

# 3) Ensure timestamps are numeric
events = events.with_columns(
    pl.col("timestamp").cast(pl.Int64, strict=False)
)
ip = item_properties_raw.with_columns(
    pl.col("timestamp").cast(pl.Int64, strict=False)
)

# 4) Preview datasets
print(events.head(), "\n")
print(ip.head(), "\n")
print(cat_tree.head())


shape: (5, 5)
┌───────────────┬───────────┬───────┬────────┬───────────────┐
│ timestamp     ┆ visitorid ┆ event ┆ itemid ┆ transactionid │
│ ---           ┆ ---       ┆ ---   ┆ ---    ┆ ---           │
│ i64           ┆ i64       ┆ str   ┆ i64    ┆ str           │
╞═══════════════╪═══════════╪═══════╪════════╪═══════════════╡
│ 1433220000000 ┆ 257597    ┆ view  ┆ 355908 ┆ null          │
│ 1433220000000 ┆ 992329    ┆ view  ┆ 248676 ┆ null          │
│ 1433220000000 ┆ 111016    ┆ view  ┆ 318965 ┆ null          │
│ 1433220000000 ┆ 483717    ┆ view  ┆ 253185 ┆ null          │
│ 1433220000000 ┆ 951259    ┆ view  ┆ 367447 ┆ null          │
└───────────────┴───────────┴───────┴────────┴───────────────┘ 

shape: (5, 4)
┌───────────────┬────────┬────────────┬─────────────────────────────────┐
│ timestamp     ┆ itemid ┆ property   ┆ value                           │
│ ---           ┆ ---    ┆ ---        ┆ ---                             │
│ i64           ┆ i64    ┆ str        ┆ str            

In [ ]:
# --- Convert timestamp to datetime ---
events = events.with_columns(
    pl.from_epoch(pl.col("timestamp") / 1000, time_unit="s").alias("datetime")
)

item_properties_raw = item_properties_raw.with_columns(
    pl.from_epoch(pl.col("timestamp") / 1000, time_unit="s").alias("datetime")
)

# --- Handle nulls ---
events = events.fill_null("unknown")  # Replace null transactionid with 'unknown'
item_properties_raw = item_properties_raw.fill_null("unknown")

# --- Clean 'value' column ---
# 1. Remove leading "n" from prices and convert to float where possible
item_properties_raw = item_properties_raw.with_columns(
    pl.col("value").str.strip_chars().alias("value_clean")
)

item_properties_raw = item_properties_raw.with_columns(
    pl.when(pl.col("value_clean").str.starts_with("n"))
    .then(pl.col("value_clean").str.strip_chars("n").cast(pl.Float64, strict=False))
    .otherwise(pl.col("value_clean"))
    .alias("value_parsed")
)

# --- Remove duplicates ---
events = events.unique()
item_properties_raw = item_properties_raw.unique()

# --- Optional: Filter unrealistic timestamps (if needed) ---
events = events.filter(pl.col("datetime").dt.year().is_between(2014, 2020))
item_properties_raw = item_properties_raw.filter(pl.col("datetime").dt.year().is_between(2014, 2020))

# --- Final clean datasets ---
print(events.head())
print(item_properties_raw.head())

: 

: 

: 

In [ ]:
# Make sure IDs are numeric
cat_tree = category_tree(in).with_columns([
    pl.col("categoryid").cast(pl.Int64, strict=False),
    pl.col("parentid").cast(pl.Int64, strict=False)
])

# Drop duplicates (based on all columns)
cat_tree = category_tree(in).unique()

# Remove rows with nulls in critical columns
cat_tree = category_tree(in).drop_nulls(subset=["categoryid", "parentid"])

# Preview
print(cat_tree.head())
print(f"Rows after cleaning: {cat_tree.shape[0]}")
